In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [3]:
def load_and_inspect_data(csv_path):
    """Load CSV (auto-detects delimiter) and perform initial inspection"""
    print("=" * 70)
    print("STEP 1: LOADING AND INSPECTING DATA")
    print("=" * 70)

    # Auto-detect delimiter using csv.Sniffer
    import csv
    with open(csv_path, 'r', encoding='utf-8') as f:
        sample = f.read(2048)
        sniffer = csv.Sniffer()
        try:
            dialect = sniffer.sniff(sample)
            sep = dialect.delimiter
        except csv.Error:
            sep = ','  # fallback if detection fails

    print(f"Detected delimiter: '{sep}'")

    # Load data
    df = pd.read_csv(csv_path, sep=sep)

    # Basic inspection
    print(f"\nDataset Shape: {df.shape[0]} rows × {df.shape[1]} columns")
    print(f"\nColumn Names:\n{df.columns.tolist()}")

    print(f"\nData Types:")
    print(df.dtypes)

    print(f"\nFirst 3 rows:")
    print(df.head(3))

    print(f"\nBasic Statistics:")
    print(df.describe())

    return df

In [4]:
# ==================== STEP 2: Handle Missing Values ====================

def analyze_missing_values(df):
    """Analyze missing values in detail"""
    print("\n" + "=" * 70)
    print("STEP 2: MISSING VALUES ANALYSIS")
    print("=" * 70)

    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100

    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing_Count': missing.values,
        'Missing_Percentage': missing_pct.values
    })
    missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

    if len(missing_df) > 0:
        print("\nColumns with Missing Values:")
        print(missing_df.to_string(index=False))
    else:
        print("\nNo missing values found!")

    return missing_df

def handle_missing_values(df):
    """Clean missing values based on column type"""
    print("\n" + "=" * 70)
    print("STEP 3: HANDLING MISSING VALUES")
    print("=" * 70)

    df_clean = df.copy()

    # 1. Remove rows without image URLs (critical for CNN)
    before = len(df_clean)
    df_clean = df_clean.dropna(subset=['image_url'])
    print(f"\n1. Removed {before - len(df_clean)} rows without image URLs")

    # 2. Remove rows without product titles (needed for categorization)
    before = len(df_clean)
    df_clean = df_clean.dropna(subset=['title'])
    print(f"2. Removed {before - len(df_clean)} rows without titles")

    # 3. Handle missing prices - fill with median by brand
    if df_clean['price'].isnull().sum() > 0:
        print(f"3. Found {df_clean['price'].isnull().sum()} missing prices")

        # Fill with brand median
        df_clean['price'] = df_clean.groupby('brand')['price'].transform(
            lambda x: x.fillna(x.median())
        )

        # If still missing, fill with overall median
        df_clean['price'].fillna(df_clean['price'].median(), inplace=True)
        print(f"   Filled missing prices with brand/overall median")

    # 4. Handle missing ratings - fill with brand average
    if df_clean['rating'].isnull().sum() > 0:
        print(f"4. Found {df_clean['rating'].isnull().sum()} missing ratings")

        df_clean['rating'] = df_clean.groupby('brand')['rating'].transform(
            lambda x: x.fillna(x.mean())
        )

        # If still missing, fill with overall mean
        df_clean['rating'].fillna(df_clean['rating'].mean(), inplace=True)
        print(f"   Filled missing ratings with brand/overall average")

    print(f"\nFinal dataset size: {len(df_clean)} rows")

    return df_clean

In [5]:
# ==================== STEP 4: Data Validation ====================

def validate_data(df):
    """Validate data quality and constraints"""
    print("\n" + "=" * 70)
    print("STEP 4: DATA VALIDATION")
    print("=" * 70)

    issues = []

    # 1. Check for duplicates
    duplicates = df.duplicated(subset=['product_id']).sum()
    if duplicates > 0:
        issues.append(f"Found {duplicates} duplicate product_ids")
        print(f"\n⚠️  WARNING: {duplicates} duplicate product IDs found")
    else:
        print(f"\n✓ No duplicate product IDs")

    # 2. Validate URLs
    invalid_urls = df[~df['image_url'].str.contains('http', na=False)].shape[0]
    if invalid_urls > 0:
        issues.append(f"Found {invalid_urls} invalid image URLs")
        print(f"⚠️  WARNING: {invalid_urls} invalid image URLs")
    else:
        print(f"✓ All image URLs are valid")

    # 3. Check price range
    negative_prices = df[df['price'] < 0].shape[0]
    if negative_prices > 0:
        issues.append(f"Found {negative_prices} negative prices")
        print(f"⚠️  WARNING: {negative_prices} negative prices")
    else:
        print(f"✓ All prices are non-negative")

    # Identify outliers
    q1 = df['price'].quantile(0.25)
    q3 = df['price'].quantile(0.75)
    iqr = q3 - q1
    outliers = df[(df['price'] < q1 - 1.5*iqr) | (df['price'] > q3 + 1.5*iqr)].shape[0]
    print(f"   Found {outliers} price outliers (outside 1.5×IQR)")

    # 4. Validate ratings
    invalid_ratings = df[(df['rating'] < 0) | (df['rating'] > 5)].shape[0]
    if invalid_ratings > 0:
        issues.append(f"Found {invalid_ratings} invalid ratings")
        print(f"⚠️  WARNING: {invalid_ratings} ratings outside 0-5 range")
    else:
        print(f"✓ All ratings are in valid range (0-5)")

    # 5. Check title length
    short_titles = df[df['title'].str.len() < 10].shape[0]
    if short_titles > 0:
        print(f"   Found {short_titles} very short titles (<10 chars)")

    return issues

def remove_duplicates(df):
    """Remove duplicate products"""
    print("\n" + "=" * 70)
    print("STEP 5: REMOVING DUPLICATES")
    print("=" * 70)

    before = len(df)

    # Remove exact duplicates
    df_clean = df.drop_duplicates(subset=['product_id'], keep='first')

    print(f"\nRemoved {before - len(df_clean)} duplicate products")
    print(f"Remaining: {len(df_clean)} unique products")

    return df_clean

In [6]:
# ==================== STEP 6: Category Extraction ====================

def extract_categories(df):
    """Extract product categories from titles"""
    print("\n" + "=" * 70)
    print("STEP 6: EXTRACTING PRODUCT CATEGORIES")
    print("=" * 70)

    # Define comprehensive category keywords
    category_patterns = {
        'Footwear': [
            'shoe', 'sneaker', 'boot', 'sandal', 'slipper', 'slide',
            'clog', 'flip-flop', 'flip flop', 'footwear', 'trainer',
            'loafer', 'moccasin', 'runner', 'running shoe'
        ],
        'Bags': [
            'backpack', 'bag', 'duffel', 'tote', 'handbag', 'satchel',
            'crossbody', 'luggage', 'briefcase', 'pouch', 'purse',
            'shoulder bag', 'messenger', 'clutch', 'wallet bag'
        ],
        'Tops': [
            'shirt', 't-shirt', 'tee', 'hoodie', 'sweatshirt', 'blouse',
            'tank', 'polo', 'sweater', 'jacket', 'coat', 'cardigan',
            'blazer', 'vest', 'top'
        ],
        'Bottoms': [
            'pant', 'jean', 'short', 'trouser', 'legging', 'skirt',
            'jogger', 'sweatpant', 'cargo'
        ],
        'Accessories': [
            'watch', 'belt', 'hat', 'cap', 'scarf', 'glove',
            'sunglasses', 'glasses', 'jewelry', 'jewellery', 'earring',
            'necklace', 'bracelet', 'ring', 'umbrella', 'headband',
            'tie', 'bowtie', 'suspender'
        ],
        'Socks': [
            'sock', 'hosiery', 'stocking'
        ],
        'Underwear': [
            'underwear', 'brief', 'boxer', 'bra', 'panties', 'lingerie',
            'undergarment', 'trunk'
        ],
        'Sportswear': [
            'athletic', 'sport', 'gym', 'fitness', 'training',
            'performance', 'active'
        ]
    }

    def categorize_product(title):
        """Categorize a single product based on title"""
        title_lower = title.lower()

        # Check each category (order matters for specificity)
        for category, keywords in category_patterns.items():
            if any(keyword in title_lower for keyword in keywords):
                return category

        return 'Other'

    # Apply categorization
    df['product_category'] = df['title'].apply(categorize_product)

    # Display results
    print("\nCategory Distribution:")
    category_counts = df['product_category'].value_counts()

    for category, count in category_counts.items():
        percentage = (count / len(df)) * 100
        print(f"  {category:15s}: {count:5d} ({percentage:5.2f}%)")

    print(f"\nTotal Products: {len(df)}")

    return df

In [7]:
# ==================== STEP 7: Text Cleaning ====================

def clean_text_fields(df):
    """Clean and standardize text fields"""
    print("\n" + "=" * 70)
    print("STEP 7: CLEANING TEXT FIELDS")
    print("=" * 70)

    df_clean = df.copy()

    # Clean titles
    print("\n1. Cleaning product titles...")
    df_clean['title'] = df_clean['title'].str.strip()
    df_clean['title'] = df_clean['title'].str.replace(r'\s+', ' ', regex=True)
    df_clean['title'] = df_clean['title'].str.replace(r'[^\w\s\-\.]', ' ', regex=True)

    # Clean brand names
    print("2. Standardizing brand names...")
    df_clean['brand'] = df_clean['brand'].str.strip()
    df_clean['brand'] = df_clean['brand'].str.upper()

    # Fix common brand name issues
    brand_mapping = {
        'JANSPORT': 'JANSPORT',
        'JAN SPORT': 'JANSPORT',
        'ADIDAS': 'ADIDAS',
        'NIKE': 'NIKE',
        'PUMA': 'PUMA',
        'SKECHERS': 'SKECHERS',
        'SKETCHERS': 'SKECHERS',
    }
    df_clean['brand'] = df_clean['brand'].replace(brand_mapping)

    print(f"   Found {df_clean['brand'].nunique()} unique brands")

    return df_clean

In [8]:
# ==================== STEP 8: Feature Engineering ====================

def add_features(df):
    """Add useful features for analysis"""
    print("\n" + "=" * 70)
    print("STEP 8: FEATURE ENGINEERING")
    print("=" * 70)

    df_enhanced = df.copy()

    # 1. Price bins
    df_enhanced['price_category'] = pd.cut(
        df_enhanced['price'],
        bins=[0, 50, 100, 200, 500, float('inf')],
        labels=['Budget', 'Economy', 'Mid-Range', 'Premium', 'Luxury']
    )
    print("\n1. Added price_category feature")

    # 2. Rating bins
    df_enhanced['rating_category'] = pd.cut(
        df_enhanced['rating'],
        bins=[0, 3.5, 4.0, 4.5, 5.0],
        labels=['Low', 'Medium', 'High', 'Excellent']
    )
    print("2. Added rating_category feature")

    # 3. Title length
    df_enhanced['title_length'] = df_enhanced['title'].str.len()
    print("3. Added title_length feature")

    # 4. Word count
    df_enhanced['word_count'] = df_enhanced['title'].str.split().str.len()
    print("4. Added word_count feature")

    # 5. Has discount (if price seems like a discount)
    df_enhanced['is_discounted'] = df_enhanced['price'] % 10 == 9
    print("5. Added is_discounted indicator")

    return df_enhanced

In [9]:
# ==================== STEP 9: Export Clean Data ====================

def export_clean_data(df, output_dir='cleaned_data'):
    """Export cleaned dataset"""
    print("\n" + "=" * 70)
    print("STEP 9: EXPORTING CLEANED DATA")
    print("=" * 70)

    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)

    # 1. Export full cleaned dataset
    csv_path = output_path / 'fashion_products_cleaned.csv'
    df.to_csv(csv_path, index=False)
    print(f"\n1. Saved cleaned dataset: {csv_path}")
    print(f"   Shape: {df.shape}")

    # 2. Export by category
    category_dir = output_path / 'by_category'
    category_dir.mkdir(exist_ok=True)

    for category in df['product_category'].unique():
        cat_df = df[df['product_category'] == category]
        cat_path = category_dir / f'{category.lower()}_products.csv'
        cat_df.to_csv(cat_path, index=False)

    print(f"2. Saved category-specific files to: {category_dir}")

    # 3. Export data summary
    summary_path = output_path / 'data_summary.txt'
    with open(summary_path, 'w') as f:
        f.write("FASHION DATASET SUMMARY\n")
        f.write("=" * 70 + "\n\n")

        f.write(f"Total Products: {len(df)}\n")
        f.write(f"Unique Brands: {df['brand'].nunique()}\n")
        f.write(f"Categories: {df['product_category'].nunique()}\n\n")

        f.write("Category Distribution:\n")
        f.write(df['product_category'].value_counts().to_string())
        f.write("\n\n")

        f.write("Price Statistics:\n")
        f.write(df['price'].describe().to_string())
        f.write("\n\n")

        f.write("Rating Statistics:\n")
        f.write(df['rating'].describe().to_string())
        f.write("\n\n")

        f.write("Top 10 Brands:\n")
        f.write(df['brand'].value_counts().head(10).to_string())

    print(f"3. Saved data summary: {summary_path}")

    return output_path

In [10]:
def run_preprocessing_pipeline(csv_path):
    """Run complete preprocessing pipeline"""
    print("\n" + "=" * 70)
    print("FASHION DATASET PREPROCESSING PIPELINE")
    print("=" * 70)

    # Step 1: Load data
    df = load_and_inspect_data(csv_path)

    # Step 2-3: Handle missing values
    missing_analysis = analyze_missing_values(df)
    df = handle_missing_values(df)

    # Step 4-5: Validate and clean
    issues = validate_data(df)
    df = remove_duplicates(df)

    # Step 6: Extract categories
    df = extract_categories(df)

    # Step 7: Clean text
    df = clean_text_fields(df)

     # Step 8: Add features
    df = add_features(df)

    # Step 9: Export
    output_path = export_clean_data(df)

    # Final summary
    print("\n" + "=" * 70)
    print("PREPROCESSING COMPLETE!")
    print("=" * 70)
    print(f"\nFinal Dataset Statistics:")
    print(f"  Total Products: {len(df)}")
    print(f"  Categories: {df['product_category'].nunique()}")
    print(f"  Brands: {df['brand'].nunique()}")
    print(f"  Price Range: {df['price'].min():.2f} - {df['price'].max():.2f} AED")
    print(f"  Average Rating: {df['rating'].mean():.2f}")

    print(f"\n✓ Cleaned data saved to: {output_path}")
    print(f"\nNext Step: Run image download script")

    return df

In [11]:
if __name__ == "__main__":
    # Run the complete pipeline
    csv_file = 'products.csv'  # Change to your file path

    cleaned_df = run_preprocessing_pipeline(csv_file)

    print("\n" + "=" * 70)
    print("Ready for image download and model training!")
    print("=" * 70)


FASHION DATASET PREPROCESSING PIPELINE
STEP 1: LOADING AND INSPECTING DATA
Detected delimiter: ','

Dataset Shape: 13156 rows × 8 columns

Column Names:
['product_id', 'brand', 'title', 'price', 'category', 'rating', 'image_url', 'product_url']

Data Types:
product_id      object
brand           object
title           object
price          float64
category        object
rating         float64
image_url       object
product_url     object
dtype: object

First 3 rows:
   product_id     brand                                              title  \
0  B08YRWN3WB  JANSPORT  Big Student Large laptop backpack Black EK0A5B...   
1  B08YRXFZZM  JANSPORT                                Superbreak Day Pack   
2  B09Q2PQ7ZB   BAODINI  Mini Travel Umbrella With Case Small Compact U...   

    price    category  rating  \
0  189.00  New season     4.7   
1  119.00  New season     4.6   
2   17.79  New season     4.2   

                                           image_url  \
0  https://m.media-amazon.

In [26]:
# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# ==================== 1. CATEGORY DISTRIBUTION ====================

def plot_category_distribution(df, save_path='plots'):
    """Visualize category distribution"""
    Path(save_path).mkdir(exist_ok=True)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Bar chart
    category_counts = df['product_category'].value_counts()
    axes[0].barh(category_counts.index, category_counts.values, color='steelblue')
    axes[0].set_xlabel('Number of Products', fontsize=12, fontweight='bold')
    axes[0].set_title('Product Count by Category', fontsize=14, fontweight='bold')
    axes[0].grid(axis='x', alpha=0.3)

    # Add count labels
    for i, v in enumerate(category_counts.values):
        axes[0].text(v + 50, i, str(v), va='center', fontweight='bold')

    # Pie chart
    colors = plt.cm.Set3(range(len(category_counts)))
    wedges, texts, autotexts = axes[1].pie(
        category_counts.values,
        labels=category_counts.index,
        autopct='%1.1f%%',
        colors=colors,
        startangle=90
    )
    axes[1].set_title('Category Distribution', fontsize=14, fontweight='bold')

    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

    plt.tight_layout()
    plt.savefig(f'{save_path}/category_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()

# ==================== 2. CLASS IMBALANCE ====================

def plot_class_balance(df, save_path='plots'):
    """Check for class imbalance issues"""
    Path(save_path).mkdir(exist_ok=True)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    category_counts = df['product_category'].value_counts()

    # 1. Class balance visualization
    axes[0].bar(range(len(category_counts)), category_counts.values, color='steelblue')
    axes[0].set_xticks(range(len(category_counts)))
    axes[0].set_xticklabels(category_counts.index, rotation=45, ha='right')
    axes[0].set_ylabel('Number of Samples', fontsize=11, fontweight='bold')
    axes[0].set_title('Class Balance Check', fontsize=13, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)

    # Add threshold line for minimum recommended samples
    min_recommended = 300
    axes[0].axhline(y=min_recommended, color='r', linestyle='--',
                    linewidth=2, label=f'Min Recommended: {min_recommended}')
    axes[0].legend()

    # Add value labels
    for i, v in enumerate(category_counts.values):
        axes[0].text(i, v + 50, str(v), ha='center', fontweight='bold')

    # 2. Imbalance ratio
    max_samples = category_counts.max()
    imbalance_ratios = max_samples / category_counts

    colors = ['green' if ratio < 2 else 'orange' if ratio < 5 else 'red'
              for ratio in imbalance_ratios]

    axes[1].barh(category_counts.index, imbalance_ratios.values, color=colors)
    axes[1].set_xlabel('Imbalance Ratio (Max/Current)', fontsize=11, fontweight='bold')
    axes[1].set_title('Class Imbalance Analysis', fontsize=13, fontweight='bold')
    axes[1].axvline(x=2, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='Moderate (2x)')
    axes[1].axvline(x=5, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Severe (5x)')
    axes[1].legend()
    axes[1].grid(axis='x', alpha=0.3)
    axes[1].invert_yaxis()

    # Add value labels
    for i, v in enumerate(imbalance_ratios.values):
        axes[1].text(v + 0.1, i, f'{v:.2f}x', va='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig(f'{save_path}/class_balance.png', dpi=300, bbox_inches='tight')
    plt.close()

    # Print warnings
    print("\n" + "=" * 70)
    print("Class Balance Report:")
    print("-" * 70)
    severe_imbalance = imbalance_ratios[imbalance_ratios > 5]
    if len(severe_imbalance) > 0:
        print("Categories with severe imbalance (>5x):")
        for cat in severe_imbalance.index:
            print(f"  - {cat}: {category_counts[cat]} samples ({imbalance_ratios[cat]:.2f}x)")
    else:
        print("✓ Class balance is acceptable (all classes within 5x of largest)")
    print("=" * 70)

# ==================== 3. CORRELATION HEATMAP ====================

def plot_correlation_heatmap(df, save_path='plots'):
    """Plot correlation heatmap for numerical features"""
    Path(save_path).mkdir(exist_ok=True)

    # Select numerical columns
    numeric_cols = ['price', 'rating', 'title_length', 'word_count']
    numeric_cols = [col for col in numeric_cols if col in df.columns]

    if len(numeric_cols) < 2:
        print("Not enough numerical features for correlation analysis")
        return

    corr_matrix = df[numeric_cols].corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=1, fmt='.3f',
                cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Heatmap', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'{save_path}/correlation_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

# ==================== MAIN EXECUTION ====================

def run_essential_visualizations(csv_path, plot_dir='plots'):
    # Load cleaned data
    print("Loading dataset...")
    df = pd.read_csv(csv_path)
    print(f"✓ Loaded {len(df):,} records")
    print(f"✓ Found {df['product_category'].nunique()} categories\n")

    # Generate visualizations
    print("Generating visualizations...\n")

    plot_category_distribution(df, plot_dir)
    plot_class_balance(df, plot_dir)
    plot_correlation_heatmap(df, plot_dir)

    print("\n" + "=" * 70)
    print("VISUALIZATION COMPLETE!")
    print("=" * 70)
    print(f"\nAll plots saved to: {plot_dir}/")
    print("  - category_distribution.png")
    print("  - class_balance.png")
    print("  - correlation_heatmap.png")

# ==================== USAGE ====================

if __name__ == "__main__":
    # Run on cleaned data
    csv_file = 'cleaned_data/fashion_products_cleaned.csv'

    run_essential_visualizations(csv_file, plot_dir='plots')

Loading dataset...
✓ Loaded 11,368 records
✓ Found 9 categories

Generating visualizations...


Class Balance Report:
----------------------------------------------------------------------
Categories with severe imbalance (>5x):
  - Socks: 297 samples (8.10x)
  - Underwear: 159 samples (15.13x)
  - Sportswear: 101 samples (23.81x)

VISUALIZATION COMPLETE!

All plots saved to: plots/
  - category_distribution.png
  - class_balance.png
  - correlation_heatmap.png
